In [3]:
from anet_api.db import Team, Athlete, Meet, TeamCreate
from anet_api.db.utils import create_item
import requests as r
from sqlmodel import select, Session
from anet_api.db.database import SessionLocal
import json

In [4]:
api_url = 'http://127.0.0.1:8000/'

In [10]:
metro_teams = [
    dict(anet_id=489, name='Ballard'),
    dict(anet_id=415, name='Bishop Blanchet'),
    dict(anet_id=416, name='Chief Sealth'),
    dict(anet_id=417, name='Cleveland'),
    dict(anet_id=418, name='Eastside Catholic'),
    dict(anet_id=492, name='Franklin'),
    dict(anet_id=483, name='Garfield'),
    dict(anet_id=419, name='Holy Names Academy'),
    dict(anet_id=420, name='Ingraham'),
    dict(anet_id=421, name='Lakeside'),
    dict(anet_id=78877, name='Lincoln'),
    dict(anet_id=422, name='Nathan Hale'),
    dict(anet_id=423, name='O\'Dea'),
    dict(anet_id=424, name='Rainier Beach'),
    dict(anet_id=498, name='Roosevelt'),
    dict(anet_id=425, name='Seattle Prep'),
    dict(anet_id=426, name='West Seattle'),
    dict(anet_id=309,name='Seattle Academy'),
    dict(anet_id=414,name='Bainbridge')
]

In [11]:
for team in metro_teams:
    r.post(api_url + 'db/addTeam',data=json.dumps(team))

In [12]:
def get_all_teams(session: Session):
    stmt = select(Team)
    return session().exec(stmt).all()

def insert_meets_and_athletes(params):
    q = r.get(api_url + 'anet/getTeam', params = params)
    meets = q.json()['schedule']
    roster = q.json()['roster']
    for meet in meets:
        response = r.post(api_url + 'db/addMeet',data=json.dumps(meet))
        if response.status_code != 200:
            print(meet['meet'], response.text)
    for athlete in roster:
        response = r.post(api_url + 'db/addAthlete', data=json.dumps(athlete))
        if response.status_code != 200:
            print(athlete['anet_id'], response.text)
    return

In [14]:
season = 2022
sport = 'xc'
for team in get_all_teams(SessionLocal):
    print(team)
    params = dict(team_id = team.anet_id, season = season, sport = sport)
    insert_meets_and_athletes(params)

name='Test High School' created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894) last_edited=datetime.datetime(2024, 12, 19, 0, 3, 45, 967000) anet_id=1 id=1
name='Garfield' created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894) last_edited=datetime.datetime(2024, 12, 19, 8, 7, 32, 184226) anet_id=493 id=2
17657892 {"detail":"Athlete already exists"}
21370434 {"detail":"Athlete already exists"}
21370415 {"detail":"Athlete already exists"}
23753866 {"detail":"Athlete already exists"}
21370417 {"detail":"Athlete already exists"}
17657871 {"detail":"Athlete already exists"}
23753871 {"detail":"Athlete already exists"}
23753856 {"detail":"Athlete already exists"}
17657868 {"detail":"Athlete already exists"}
22211896 {"detail":"Athlete already exists"}
22211747 {"detail":"Athlete already exists"}
19608500 {"detail":"Athlete already exists"}
17657897 {"detail":"Athlete already exists"}
17657886 {"detail":"Athlete already exists"}
17657894 {"detail":"Athlete already exists"}
2375386

In [12]:
from sqlalchemy import extract
def get_all_meets_by_season(session: Session, season: int):
    stmt = select(Meet).where(extract('year', Meet.date) == season)
    return session.exec(stmt).all()

In [13]:

meets = get_all_meets_by_season(SessionLocal(),2023)




In [8]:
meets

[Meet(venue='Steens Mountain ', city='FrenchGlen', zipcode=97736, id=2, anet_id=225111, last_edited=datetime.datetime(2024, 9, 27, 16, 37, 55, 16434), created_at=datetime.datetime(2024, 9, 27, 16, 34, 45, 579588), meet='Steens Mountain Running Camp Jan Yriarte Uphill 5K ', address=None, state='OR', date=datetime.date(2023, 7, 28)),
 Meet(venue='Lake Wilderness State Park', city='Maple Valley', zipcode=98038, id=3, anet_id=222038, last_edited=datetime.datetime(2024, 9, 27, 16, 37, 55, 25497), created_at=datetime.datetime(2024, 9, 27, 16, 34, 45, 579588), meet='Tahoma Coed Relays (Jamboree)', address='22500 SE 248th Street ', state='WA', date=datetime.date(2023, 9, 9)),
 Meet(venue='Lincoln Park', city='Seattle', zipcode=98136, id=4, anet_id=229674, last_edited=datetime.datetime(2024, 9, 27, 16, 37, 55, 32062), created_at=datetime.datetime(2024, 9, 27, 16, 34, 45, 579588), meet='Metro XC @ Lincoln Park WS/Ballard/Garfield/CLeveland/Chief Sealth/East Side Catholic ', address='8011 Fauntle

In [14]:
def insert_new_result(meet_id: int):
    params = dict(sport = 'xc', meet_id = meet_id)
    s = r.get(api_url + 'anet/getResults',params=params)
    s.json()

    all_results = []
    for race in s.json()['races']:
        all_results += race['results']

    for result in all_results:
        response = r.post(api_url + 'db/addResult', data = json.dumps(result))
        if response.status_code == 422:
            print(response.text)
    return "All done!"

In [15]:
for meet in meets:
    insert_new_result(meet.anet_id)

{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","anet_athlete_id"],"msg":"Input should be a valid integer","input":null}]}
{"detail":[{"type":"int_type","loc":["body","ane

: 

In [26]:
session = SessionLocal()
team = Team(anet_id=111, name="New Team")
athlete = Athlete(anet_id=222, first_name="John", last_name="Doe", gender="F")
meet = Meet(anet_id=333, name="New Meet", venue="Place", date="2023-01-01")

session.add(team)
session.add(athlete)
session.add(meet)
session.commit()

In [11]:
import requests as r
url = 'https://www.athletic.net/api/v1/Event/Events'
data = dict(
        start='2022-11-01',
        end='2022-11-30',
        levelMask=4,
        sportMask=2,
        state='WA',
        country='us',
    )

res = r.post(url,json=data)

In [12]:
res.json()

{'events': [{'IDMeet': 224294,
   'Sport': 'XC',
   'MeetName': 'Forks HS League Meet (CANCELED)',
   'SeasonId': 2023,
   'StartDate': '0001-01-01T00:00:00',
   'EndDate': '2023-09-19T00:00:00',
   'Virtual': False,
   'LocationName': 'Olympic Natural Resources Center',
   'StreetAddress': '1455 S Forks Ave',
   'City': 'Forks',
   'PostalCode': '98331',
   'State': 'WA',
   'OwnerID': 364,
   'CalendarLock': 0,
   'UCalendarLock': 0,
   'RegEnd': None,
   'HasResults': 0,
   'LevelMask': 6,
   'HostName': 'Forks',
   'MascotUrl': '//lh3.googleusercontent.com/AUH2hZZ7KFJd1HnvbCxwUWO5zaQuN2YUpzeckUG9V9bd6Y6BpCO4ajPePXdkY0zjbVNaEfaKkhQsHPFr08_kMw',
   'Lat': 47.9330028,
   'Long': -124.3944981,
   'score': 0.0,
   'CalCount': 1,
   'OffDays': '{}',
   'Data': None,
   'rsUrl': None,
   'LiveID': None,
   'LivePublished': True,
   'VideoURL': None,
   'Website': '',
   'Country': 'US'},
  {'IDMeet': 225599,
   'Sport': 'XC',
   'MeetName': 'Removed',
   'SeasonId': 2023,
   'StartDate': 